In [10]:
from langchain import LLMChain
from langchain.document_loaders import DirectoryLoader,PyPDFLoader,JSONLoader,TextLoader
from langchain.document_loaders import Docx2txtLoader,UnstructuredHTMLLoader,CSVLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate
from langchain.llms.base import LLM
from langchain.vectorstores import Qdrant
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModel, AutoConfig
from typing import Any, Dict, List, Mapping, Optional, Tuple, Union
import torch
import re
import glob
import os

In [2]:
def cut_sent(para):
    para = re.sub('([。！？\?])([^”’])', r"\1\n\2", para)  # 单字符断句符
    para = re.sub('(\.{6})([^”’])', r"\1\n\2", para)  # 英文省略号
    para = re.sub('(\…{2})([^”’])', r"\1\n\2", para)  # 中文省略号
    para = re.sub('([。！？\?][”’])([^，。！？\?])', r'\1\n\2', para)
    # 如果双引号前有终止符，那么双引号才是句子的终点，把分句符\n放到双引号后，注意前面的几句都小心保留了双引号
    para = para.rstrip()  # 段尾如果有多余的\n就去掉它
    # 很多规则中会考虑分号;，但是这里我把它忽略不计，破折号、英文双引号等同样忽略，需要的再做些简单调整即可。
    return len(para.split("\n"))


In [4]:
cut_sent('LangChain是由Harrison Chase创建的Python库，提供了开箱即用的支持，用于使用LLM（Large Language Models）构建自然语言处理应用程序。您可以连接到各种数据和计算源，并构建在特定领域数据源、私有存储库等上执行自然语言处理任务的应用程序。')

2

In [5]:
def data_loader(file):
    loader = None
    if os.path.splitext(file)[-1]=='.pdf':
        loader = PyPDFLoader(file)
    elif os.path.splitext(file)[-1] in ['.oc','.docx']:
        loader = Docx2txtLoader(file)
    elif os.path.splitext(file)[-1]=='.html':
         loader = UnstructuredHTMLLoader(file)
    elif os.path.splitext(file)[-1]=='.json':
         loader = JSONLoader(file)
    elif os.path.splitext(file)[-1]=='.csv':
         loader = CSVLoader(file)
    elif os.path.splitext(file)[-1] in ['.txt','.data','.dat']:
         loader = TextLoader(file)  
    return loader

In [17]:
def data_import(data_path):
    # 加载文件夹中的所有txt类型的文件
    file_list = glob.glob(os.path.join(data_path,'*'))

    # 初始化加载器
    text_splitter = CharacterTextSplitter(        
        separator = "\n",
        chunk_size = 10,
        chunk_overlap  = 1,
        length_function = cut_sent,
        )
    
    ## 加载向量化模型
    embeddings = HuggingFaceEmbeddings(model_name='/root/ChatBot/QA/text2vec_cmed')
    
    for file in file_list:
        try:
            loader = data_loader(file)
            documents = loader.load()
        except:
            print('文件%s读取失败' % file)
            continue
        # 切割加载的 document
        split_docs = text_splitter.split_documents(documents)

        # 将 document 通过 embeddings 对象计算 embedding 向量信息并临时存入 Qdrant 向量数据库，用于后续匹配查询
        vqdrant = Qdrant.from_documents(
        split_docs, embeddings, 
        path="./tmp/local_qdrant",
        collection_name="my_documents")


In [11]:
data_path = 'data'
file_list = glob.glob(os.path.join(data_path,'*'))

In [12]:
file_list

['data/附件2 中山大学关于印发《中山大学高校基本科研业务费管理实施细则》通知.pdf']

In [13]:
loader = data_loader(file_list[0])
documents = loader.load()
documents

[Document(page_content='中山大学高校基本科研业务费管理实施细\n则 \n（2021第4次校党委常委会议通过，根据 2021第18次\n校党委常委会议 决定修正）  \n \n第一章   总则  \n第一条  为进一步加强学校高校基本科研业务费管理，\n提高资金使用效益，根据《中央高校基本科研业务费管理办\n法》 （财教 〔2016〕277号）以及国家科研项目资金管理改革\n和赋予基层单位科研管理自主权等相关改革精神和文件要\n求，结合学校实际，制定本细则。  \n第二条  学校高校基本科研业务费按照 “统筹规划与自\n主安排相结合、公开遴选、按需分配、择优支持 ”的原则，重\n点支持各学院（直属系）、附属医院及科研机构（以下简称\n二级单位） 的中青年教师提升基本科研能力、 建设科研团队、\n开展关键核心技术创新与前沿新兴交叉学科研究。  \n第三条  学校从每年高校基本科研业务费总量中安排\n20%作为绩效奖励经费，以业绩成果为导向，激励 中青年教\n师发展。  \n第四条  科学研究院统筹学校高 校基本科研业务费管\n理工作，负责组织项目评审、预算编制、预算执行和绩效考\n评工作等。发展规划办公室负责按财政与教育主管部门要求\n组织项目资金预算申报，监督检查项目的预算执行与资金使\n用效益等。财务处负责财务管理、会计核算、提供财务数据\n及财务报表审核等。人力资源管理处负责绩效奖励发放等。  ', metadata={'source': 'data/附件2 中山大学关于印发《中山大学高校基本科研业务费管理实施细则》通知.pdf', 'page': 0}),
 Document(page_content='各二级单位负责具体项目的组织实施与管理，为项目\n的执行提供条件保障，承担监管责任等。  \n项目负责人是项目实施和资金使用的直接责任人，负责\n按申请书或任务书要求组织开展研究，依法、据实编制项目\n预算和决算，按预算规范、及时和高效使用经费，接受上级\n和学校相关部门的监督检查，并对任务执行进度、质量及经\n费使用的合法性、合规性、合理性、真实性和有效性承担行\n政、经济和法律责任。  \n第五条  本细则适用于学校所有 受高校基本科研业务\n费资助的二级单位、团队及个人。  \n \n第二章   项目管理  \n第六条  高校基本科

In [18]:
text_splitter = CharacterTextSplitter(        
        separator = "\n",
        chunk_size = 10,
        chunk_overlap  = 1,
        length_function = cut_sent,
        )

In [19]:
split_docs = text_splitter.split_documents(documents)
split_docs[:10]

[Document(page_content='中山大学高校基本科研业务费管理实施细\n则 \n（2021第4次校党委常委会议通过，根据 2021第18次\n校党委常委会议 决定修正）', metadata={'source': 'data/附件2 中山大学关于印发《中山大学高校基本科研业务费管理实施细则》通知.pdf', 'page': 0}),
 Document(page_content='第一章   总则  \n第一条  为进一步加强学校高校基本科研业务费管理，\n提高资金使用效益，根据《中央高校基本科研业务费管理办\n法》 （财教 〔2016〕277号）以及国家科研项目资金管理改革', metadata={'source': 'data/附件2 中山大学关于印发《中山大学高校基本科研业务费管理实施细则》通知.pdf', 'page': 0}),
 Document(page_content='法》 （财教 〔2016〕277号）以及国家科研项目资金管理改革\n和赋予基层单位科研管理自主权等相关改革精神和文件要\n求，结合学校实际，制定本细则。  \n第二条  学校高校基本科研业务费按照 “统筹规划与自\n主安排相结合、公开遴选、按需分配、择优支持 ”的原则，重', metadata={'source': 'data/附件2 中山大学关于印发《中山大学高校基本科研业务费管理实施细则》通知.pdf', 'page': 0}),
 Document(page_content='主安排相结合、公开遴选、按需分配、择优支持 ”的原则，重\n点支持各学院（直属系）、附属医院及科研机构（以下简称\n二级单位） 的中青年教师提升基本科研能力、 建设科研团队、\n开展关键核心技术创新与前沿新兴交叉学科研究。  \n第三条  学校从每年高校基本科研业务费总量中安排', metadata={'source': 'data/附件2 中山大学关于印发《中山大学高校基本科研业务费管理实施细则》通知.pdf', 'page': 0}),
 Document(page_content='第三条  学校从每年高校基本科研业务费总量中安排\n20%作为绩效奖励经费，以业绩成果为导向，激励 中青年教\n师发展。  \n第四条  科学研究院统筹学校高 校基本科研业务费管\n理工作，负责组织项目评

In [57]:
data_import(data_path)